In [1]:
from __future__ import division
from __future__ import print_function

import sys, os,gc
import tensorflow as tf
import numpy as np

sys.path.append('../')

from models.neural_network import NeuralNetwork
from models.cnn import CNN
from models.util import *

os.environ["CUDA_VISIBLE_DEVICES"] = "3"

#Seed used for all calculations of training and test point indices 
SEED = 14

Using TensorFlow backend.


In [2]:
#Define params of model
input_shape = (28,28,1)
num_classes = 10
eps=0.3


In [3]:
#Load model from disk
model_name = 'MNIST'
model_save_path = '../trained_models/' + model_name + '-model.json'
weights_save_path = '../trained_models/' + model_name + 'weights'
model = CNN(model_name=model_name, dataset='mnist', seed=SEED)
print ('Total Model Params: %d' % model.num_params)
model.load_model(model_save_path, weights_save_path) 
#epochs = 50
#model.train(epochs=epochs)
#model.save_model(model_save_path, weights_save_path)   

Total Model Params: 3330314
Loaded model from disk


In [4]:
#Model Accuracy
print ('Model Accuracy: %.5f' % (model.model.evaluate(model.test_data, model.test_labels)[1]))

6055/6055 [==============================] - 2s 333us/step
Model Accuracy: 0.99306


In [5]:
#Get training samples
num_train_samples = 100
data_indices = model.gen_rand_indices(low=0, high=model.train_data.shape[0], seed=SEED, num_samples=num_train_samples)
train_data = model.train_data[data_indices]
train_data_labels = model.train_labels[data_indices]
train_data_labels_int = np.argmax(train_data_labels, axis=1)


In [6]:
num_test_samples_per_class = 10
num_test_samples = num_classes*num_test_samples_per_class

#Generate test points
test_indices = model.gen_rand_indices_all_classes(y=model.test_labels, seed=SEED, num_samples=num_test_samples_per_class)
#Get Regular, Noisy, FGSM, BIM, and CW test points
reg_data = model.test_data[test_indices]

In [12]:
#Calculate Hessian Vector Product Matrix to be used for Influence
hvp_matrix_train = np.zeros((train_data.shape[0], model.num_params))
for idx, img_lab in enumerate(zip(train_data, train_data_labels)):
    inverse_hvp = model.get_inverse_hvp(img_lab[0], img_lab[1])
    hvp_matrix_train[idx,:] = inverse_hvp


In [7]:
#Calculate Hessian Vector Product Matrix to be used for Influence
hvp_matrix = np.zeros((reg_data.shape[0], model.num_params))
for idx, img_lab in enumerate(zip(reg_data, model.test_labels[test_indices])):
    inverse_hvp = model.get_inverse_hvp(img_lab[0], img_lab[1])
    hvp_matrix[idx,:] = inverse_hvp



In [8]:
reg_preds = model.model.predict(reg_data.reshape(-1,*input_shape))
reg_labels = preds_to_labels(reg_preds)


In [10]:
train_preds = model.model.predict(train_data.reshape(-1,*input_shape))
train_labels = preds_to_labels(train_preds)


In [9]:
#Get Gradients required for Similarity
grads_train = model.get_gradients_wrt_params(train_data, train_data_labels)
grads_reg = model.get_gradients_wrt_params(reg_data, reg_labels)


In [10]:
#Calculate Influence and Similarity Matrices
inf_matrix = np.dot(hvp_matrix, grads_train.T)
sim_matrix = np.dot(grads_reg, grads_train.T)


In [13]:
#Calculate percentage of points for which H^{-1} acts as a scaling
avg_cos = 0.0
cos_sim = list()
for i in range(num_train_samples ):
    hvp_norm=np.linalg.norm(hvp_matrix_train[i])
    train_norm = np.linalg.norm(grads_train[i])
    if hvp_norm > 0 and train_norm > 0:
        
        cos= train_norm/hvp_norm
        cos_sim.append(cos)
        
    else:
        print ('Divide by zero error, skipping point ...')

print ('Avg Cos Sim: %.5f, Var: %.5f, Median: %.5f, Min: %.5f, Max: %.5f' % (np.mean(cos_sim), np.var(cos_sim), np.median(cos_sim), np.min(cos_sim), np.max(cos_sim)))



Avg Cos Sim: 100.20286, Var: 1.24936, Median: 99.98809, Min: 99.04225, Max: 107.24023


In [20]:
#Calculate percentage of points for which H^{-1} acts as a scaling
avg_cos = 0.0
cos_sim = list()
for i in range(num_train_samples ):
    hvp_norm=np.linalg.norm(hvp_matrix_train[i])
    train_norm = np.linalg.norm(grads_train[i])
    if hvp_norm > 0 and train_norm > 0:
        a = hvp_matrix_train[i]/hvp_norm
        b = grads_train[i]/train_norm
        cos=np.dot(a,b)
        cos_sim.append(cos)
        
    else:
        print ('Divide by zero error, skipping point ...')

print ('Avg Cos Sim: %.5f, Var: %.5f, Median: %.5f, Min: %.5f, Max: %.5f' % (np.mean(cos_sim), np.var(cos_sim), np.median(cos_sim), np.min(cos_sim), np.max(cos_sim)))


Avg Cos Sim: 0.99982, Var: 0.00000, Median: 0.99986, Min: 0.99938, Max: 1.00000


In [11]:
#Get Avg Scale Constant and get ratio of test points for which (max sim == max inf)
scaling_const = 0.0
arg_max = 0.0
for j in range(num_train_samples):
    #if (np.argmax(sim_matrix[i]) == np.argmax(inf_matrix[i])):
    #    arg_max+=1.0
    scaling = list()
    for i in range(num_test_samples):
        const = sim_matrix[i,j]/inf_matrix[i,j]
        scaling.append(const)
        scaling_const+=const
    print ('Avg: %.5f, Var: %.5f, Median: %.5f, Min: %.5f, Max: %.5f' % (np.mean(scaling), np.var(scaling), np.median(scaling), np.min(scaling), np.max(scaling)))
#print ('Avg Scaling Const: %.5f, Ratio of Same Arg Max: %.5f' % ((scaling_const/(num_test_samples*num_train_samples)), (arg_max/num_test_samples)))   

Avg: -712.20943, Var: 64765896.70499, Median: 102.34315, Min: -80785.10632, Max: 113.94821
Avg: 101.31130, Var: 257.53641, Median: 99.66969, Min: 89.14463, Max: 260.03273
Avg: 102.42616, Var: 124.60414, Median: 100.85628, Min: 92.85352, Max: 197.16914
Avg: 101.60558, Var: 5.96115, Median: 101.54564, Min: 90.55167, Max: 117.91940
Avg: 100.64522, Var: 26.70253, Median: 101.08815, Min: 53.37242, Max: 112.94924
Avg: 109.36228, Var: 4167.50166, Median: 99.78024, Min: 86.38978, Max: 712.59593
Avg: 101.59134, Var: 8.45746, Median: 101.20142, Min: 97.46986, Max: 124.68836
Avg: 102.42713, Var: 75.57832, Median: 101.47824, Min: 96.86417, Max: 185.99220
Avg: 102.53529, Var: 285.00415, Median: 101.49361, Min: 85.17654, Max: 268.03638
Avg: 98.82884, Var: 6.27165, Median: 99.32780, Min: 84.17610, Max: 104.40746
Avg: 102.21326, Var: 84.88200, Median: 101.12827, Min: 94.83953, Max: 191.57206
Avg: 102.42573, Var: 11.10107, Median: 102.51864, Min: 88.64582, Max: 115.14527
Avg: 102.14648, Var: 85.35567, 

In [28]:
print ('Avg: %.5f, Var: %.5f, Median: %.5f, Min: %.5f, Max: %.5f' % (np.mean(scaling), np.var(scaling), np.median(scaling), np.min(scaling), np.max(scaling)))

Avg: 99.49839, Var: 139253.80111, Median: 100.92021, Min: -80902.50954, Max: 19914.82356


In [ ]:
hvp = list()
gt = list()
for i in range(num_train_samples ):
    hvp_norm=np.linalg.norm(hvp_matrix_train[i])
    train_norm = np.linalg.norm(grads_train[i])
    hvp.append(hvp_norm)
    gt.append(train_norm)

In [15]:
plt.scatter(hvp, gt, color='orange')
plt.ylim(np.min(gt),np.median(gt))
plt.xlim(np.min(hvp),np.median(hvp))
plt.xlabel('Transformed', fontsize=12)
plt.ylabel('Original', fontsize=12)
plt.title('Scaling: Hessian Inverse')
plt.legend(loc="lower right")
plt.tight_layout()
plt.savefig('hvp_gt.eps', format='eps', dpi=1000)
plt.show()